In [36]:
# Utilitie functions

def create_angles_window_average(angle_data, average_window, n_timesteps):
    windows = []
    for i in range(angle_data.shape[0] - n_timesteps + 1):
        windows.append(np.sum(angle_data[n_timesteps+i-1:i+average_window + n_timesteps-1], axis=0)/average_window)
    return np.array(windows)

def create_time_serie(emg_data, angle_data, n_timesteps):
    for i in range(emg_data.shape[0]):
        windows = create_sliding_windows(emg_data[i], n_timesteps)
        angles = angle_data[i][n_timesteps-1:]

        if 0 < i:
            emg_series = np.vstack((emg_series, windows))
            angle_series = np.vstack((angle_series, angles))
        else:
            emg_series = windows
            angle_series = angles

    print("EMG shape :", emg_series.shape, "Angles shape :", angle_series.shape)
    return emg_series, angle_series

# Create sliding windows of n_timesteps (data augmentation technique)
def create_sliding_windows(data, n_timesteps):
    windows = []
    print(data.shape)
    for i in range(data.shape[0] - n_timesteps + 1):
        windows.append(data[i: i + n_timesteps])
    return np.array(windows)

In [12]:

from convnet import ModelConv
import os
from os import path, listdir
from sklearn.model_selection import train_test_split
import pywt
import matplotlib.pyplot as plt

import numpy as np

from sklearn.model_selection import train_test_split

from Lib.data_preprocessing import *
from Lib.sliding_window import *

In [10]:
# Training config

TOTAL_SEQUENCES = 572
N_PHASES = 3
N_ANGLES = 18
N_ANGLES_TO_DELETE = 0
N_ELECTRODES = 7

In [22]:
# Load data from TEST_FILES_PATH

TEST_FILES_PATH = "../../../../Acquisition/Data/Dataset_avec_angles_tester"

with open(TEST_FILES_PATH + '/dataset_emg_angles.npz', 'rb') as file:
    data = np.load(file, allow_pickle=True)
    emg_data = data["emg_data"]
    angle_data = data["angle_data"]

# Pour chaques 1 sensors (emg) j'ai angle de 1 moteur??
print("EMG dataset size:", emg_data.shape)
print("Angle size:", angle_data.shape)

EMG dataset size: (572,)
Angle size: (572,)


In [37]:
# Data normalization + separate between validation and training
angle_data = angle_data
x_train, x_test, y_train, y_test = train_test_split(emg_data, angle_data, test_size=0.10, random_state=40)

print('1')
n_timesteps = 70
if not path.exists('dataset_processed.npy'):
    print('2.1')

    x_train, y_train = create_time_serie(x_train, y_train, n_timesteps)
    x_test, y_test = create_time_serie(x_test, y_test, n_timesteps)
    print('2.2')

    x_train = pywt.dwt(x_train,'haar')[0]
    x_test = pywt.dwt(x_test,'haar')[0]
    print('2.3')

    with open('dataset_processed.npy', 'wb') as f:
        np.save(f, x_train)
        np.save(f, y_train)
        np.save(f, x_test)
        np.save(f, y_test)
        print('3')


else:
    with open('dataset_processed.npy', 'rb') as f:
        x_train = np.load(f, allow_pickle=True)
        y_train = np.load(f, allow_pickle=True)
        x_test = np.load(f, allow_pickle=True)
        y_test = np.load(f, allow_pickle=True)
        print('4')


1
2.1
(958, 7)
(741, 7)
(848, 7)
(348, 7)
(1984, 7)
(1174, 7)
(1002, 7)
(580, 7)
(1356, 7)
(550, 7)
(1030, 7)
(1286, 7)
(320, 7)
(930, 7)
(424, 7)
(1065, 7)
(556, 7)
(2108, 7)
(628, 7)
(688, 7)
(946, 7)
(558, 7)
(1388, 7)
(666, 7)
(2056, 7)
(686, 7)
(884, 7)
(1858, 7)
(1310, 7)
(1312, 7)
(496, 7)
(452, 7)
(938, 7)
(602, 7)
(604, 7)
(2012, 7)
(1012, 7)
(982, 7)
(2282, 7)
(880, 7)
(1732, 7)
(898, 7)
(848, 7)
(544, 7)
(396, 7)
(688, 7)
(328, 7)
(2114, 7)
(542, 7)
(586, 7)
(674, 7)
(560, 7)
(664, 7)
(952, 7)
(828, 7)
(2775, 7)
(1120, 7)
(974, 7)
(646, 7)
(948, 7)
(1404, 7)
(880, 7)
(394, 7)
(1999, 7)
(598, 7)
(1026, 7)
(761, 7)
(2016, 7)
(1224, 7)
(1510, 7)
(1400, 7)
(973, 7)
(796, 7)
(760, 7)
(596, 7)
(1964, 7)
(612, 7)
(2618, 7)
(2382, 7)
(500, 7)
(806, 7)
(1810, 7)
(750, 7)
(892, 7)
(468, 7)
(642, 7)
(642, 7)
(524, 7)
(601, 7)
(1598, 7)
(624, 7)
(822, 7)
(776, 7)
(726, 7)
(1568, 7)
(904, 7)
(460, 7)
(512, 7)
(590, 7)
(721, 7)
(1448, 7)
(1804, 7)
(928, 7)
(418, 7)
(903, 7)
(556, 7)
(840,

In [38]:
    # train model
    
    epochs = 11
    batch_size = 500

    model = ModelConv(N_ANGLES-N_ANGLES_TO_DELETE, N_ELECTRODES, n_timesteps)
    print("x train shape :", x_train.shape, "y train shape :", y_train.shape)
    print("x test shape :", x_test.shape, "y test shape :", y_test.shape)
    model.train(x_train, y_train, x_test, y_test, epochs, batch_size)
    model.evaluate(x_test, y_test)

    model.save("conv_angle_v1_sw.h5")

x train shape : (470055, 70, 4) y train shape : (470055, 18)
x test shape : (56135, 70, 4) y test shape : (56135, 18)


2022-09-14 22:05:47.407533: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


Train on 470055 samples, validate on 56135 samples
Epoch 1/11
470055/470055 [==============================] - 145s 309us/sample - loss: 377.2329 - mae: 14.0596 - val_loss: 306.3630 - val_mae: 13.0634
Epoch 2/11
470055/470055 [==============================] - 137s 290us/sample - loss: 313.5652 - mae: 12.9307 - val_loss: 300.1451 - val_mae: 12.9393
Epoch 3/11
470055/470055 [==============================] - 137s 292us/sample - loss: 307.1037 - mae: 12.7902 - val_loss: 295.6211 - val_mae: 12.8325
Epoch 4/11
470055/470055 [==============================] - 137s 292us/sample - loss: 302.6736 - mae: 12.6908 - val_loss: 294.8547 - val_mae: 12.7836
Epoch 5/11
470055/470055 [==============================] - 1750s 4ms/sample - loss: 299.3118 - mae: 12.6119 - val_loss: 297.2305 - val_mae: 12.8611
Epoch 6/11
470055/470055 [==============================] - 136s 289us/sample - loss: 296.0841 - mae: 12.5343 - val_loss: 297.1645 - val_mae: 12.8309
Epoch 7/11
470055/470055 [========================

Accuracy: 1278.99
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]
 [  159     0     0     0   490   100    52     3     0   398   472     0
     47   182     5     0]
 [    0     0     0     0     0    18     0     0     0     0     0     0
      0     1     0     0]
 [  101     1     0    25    54    64    44     0     0   110   117     5
     20    17    22     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]
 [    0     0     0     0     0     4     0     0     0     0     0     0
      0     0     0     0]
 [  279    89     9   308  8286  6868  3229    71   294 17955  5639    12

/Users/ashbylel/opt/anaconda3/envs/metis-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ashbylel/opt/anaconda3/envs/metis-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ashbylel/opt/anaconda3/envs/metis-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [ ]:
# predict

print(model.predict(x_test).shape)
print(y_test.shape)

y_hat = model.predict(x_test)
taille = len(y_hat)

angles_hat = []
angles_test = []
angles_MSE = []

for j in range(18):
    for i in range(10000):
        angles_hat.append(y_hat[i][j])
        angles_test.append(y_test[i][j])
        angles_MSE.append((y_hat[i][j] - y_test[i][j])**2)
    #print(np.shape(angles_hat))
    #print(np.shape(angles_test))
    fig, (ax1,ax2) = plt.subplots(2,1)
    ax1.plot(angles_hat, 'r')
    ax1.plot(angles_test, 'b')
    ax2.plot(angles_MSE, 'g')
    ax2.set_xlabel("Timestep")
    ax2.set_ylabel("Squared error")
    ax2.set_title("MSE = " + str(np.mean(angles_MSE)))
    plt.show()
    angles_hat = []
    angles_test = []
    angles_MSE = []